In [ ]:
!pip install --upgrade openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 389.9/389.9 kB 7.9 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.54.5
    Uninstalling openai-1.54.5:
      Successfully uninstalled openai-1.54.5


In [ ]:
from openai import AzureOpenAI

# gets the API Key from environment variable AZURE_OPENAI_API_KEY
client = AzureOpenAI(
    # https://learn.microsoft.com/azure/ai-services/openai/reference#rest-api-versioning
    api_version="2024-08-01-preview",
    api_key="SECRET",
    # https://learn.microsoft.com/azure/cognitive-services/openai/how-to/create-resource?pivots=web-portal#create-a-resource
    azure_endpoint="SECRET",
    # Explicitly set http_client to None
    http_client=None
)

import re



In [ ]:
def chat_with_mimic4(user_input):
  completion = client.chat.completions.create(
      model="MIMIC4 NL TO SQL",  # or "MIMIC4 NL TO SQL"
      messages=[
          {"role": "user", "content": user_input, "name": "user_1"},
      ],
  )
  # Extract the query using a regular expression
  match = re.search(r"sql\n(.*)\n", completion.choices[0].message.content, re.DOTALL | re.MULTILINE)
  if match:
    query = match.group(1).strip()
    return query
  else:
    return "Query not found in response."

In [ ]:
def get_structured_input(nl_query):
  # instruction = input("Enter instruction (e.g., Get, Calculate average): ")
  # data_element = input("Enter data element (e.g., length of stay, patient IDs): ")
  # tables = input("Enter table(s) (e.g., ADMISSIONS, PATIENTS): ")
  # conditions = input("Enter conditions (optional, e.g., where admission_type = 'ELECTIVE'): ")

  # Combine into structured prompt
  prompt = f"In the conrtext of MIMIC 3 dataset, I have the following Natural Language Query: {nl_query}. Provide me with the SQL query only"
  # prompt = f"In the conrtext of MIMIC 4 dataset, I have the following Natural Language Query: Instruction: {instruction}, Data element: {data_element} from Possible Tables: {tables} and with conditions: {conditions}. Provide me with the SparQL query only"
  return prompt

In [ ]:

while True:
  user_input = get_structured_input()
  if "exit" in user_input.lower():
    break
  response = chat_with_mimic4(user_input)
  print("MIMIC4:", response)


Enter instruction (e.g., Get, Calculate average): Calculate average
Enter data element (e.g., length of stay, patient IDs): patients
Enter table(s) (e.g., ADMISSIONS, PATIENTS): admissions
Enter conditions (optional, e.g., where admission_type = 'ELECTIVE'): Have Been attended by at least 2 doctors
MIMIC4: Creating a SPARQL query for this task requires some assumptions about how the data is structured within a hypothetical RDF version of the MIMIC-IV dataset. Typically, such datasets aren't available in RDF format by default, but let's assume a possible RDF structure for this data.

Here is a basic structure of such a query:

```sparql
PREFIX ex: <http://example.org/mimic4#>

SELECT (AVG(?patientCount) as ?averagePatients)
WHERE {
  {
    SELECT ?patient
    WHERE {
      ?admission ex:hasPatient ?patient;
                 ex:hasDoctor ?doctor1;
                 ex:hasDoctor ?doctor2.

      FILTER (?doctor1 != ?doctor2)
    }
    GROUP BY ?patient
  }
  GROUP BY ?admission
}
```

### 

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab')

def extract_keywords_nltk(text):
  """Extracts keywords from text using NLTK."""
  tokens = word_tokenize(text.lower())
  stop_words = set(stopwords.words('english'))
  keywords = [word for word in tokens if word.isalnum() and word not in stop_words]
  return keywords

# Example usage
text = "This is a sample sentence with some keywords."
keywords = extract_keywords_nltk(text)
print(keywords)  # Output: ['sample', 'sentence', 'keywords']

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...


['sample', 'sentence', 'keywords']


[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [ ]:
!git clone https://github.com/junwoopark92/mimic-sparql.git

Cloning into 'mimic-sparql'...
remote: Enumerating objects: 117, done.
remote: Counting objects: 100% (117/117), done.
remote: Compressing objects: 100% (81/81), done.
remote: Total 117 (delta 56), reused 79 (delta 29), pack-reused 0 (from 0)
Receiving objects: 100% (117/117), 2.04 MiB | 6.70 MiB/s, done.
Resolving deltas: 100% (56/56), done.


In [ ]:
import json
import pandas as pd
def inspect_json_file(file_path, char_position):
  """Prints a section of the JSON file around the specified character position."""
  with open(file_path, "r") as f:
    content = f.read()
    start = max(0, char_position - 50)  # 50 characters before
    end = min(len(content), char_position + 50)  # 50 characters after
    print(f"Content around character {char_position}:")
    print(content[start:end])

# Inspect the problematic section
inspect_json_file("/content/mimic-sparql/dataset/mimicsql/mimicsql_natural/test.json", 590)

Content around character 590:
E", "DEMOGRAPHIC.\"DOB_YEAR\"", "<", "\"2060\""]}
{"key": "8ef884f5ab354b39fa29558e7dbe5606", "forma


In [ ]:
import json
import pandas as pd
#0.5
def compare_queries(predicted_query, ground_truth_query,thres=0.3):
    """Compares the predicted SparQL query with the ground truth SQL query.

    For simplicity, this function only checks if the predicted query contains
    some keywords from the ground truth query. A more robust comparison
    could be implemented using a dedicated query comparison library.
    """

    ground_truth_keywords = extract_keywords_nltk(ground_truth_query)
    predicted_keywords = extract_keywords_nltk(predicted_query)

    common_keywords = set(ground_truth_keywords) & set(predicted_keywords)

    # Print for analysis
    print("Ground truth keywords:", ground_truth_keywords)
    print("Predicted keywords:", predicted_keywords)
    print("Common keywords:", common_keywords)


    # Simple keyword-based comparison for demonstration
    if len(common_keywords) > thres*len(set(common_keywords).union(set(ground_truth_keywords)) ):
        return 1
    else:
        return 0

def evaluate_on_test_data(file_path="/content/mimic-sparql/dataset/mimicsql/mimicsql_natural/test.json",thres=0.3):
    """Evaluates `chat_with_mimic4` on the test data in `test.json`."""
    data = []
    with open("/content/mimic-sparql/dataset/mimicsql/mimicsql_natural/test.json", "r") as f:
      for line in f:
        data.append(json.loads(line))
    df = pd.DataFrame(data)
    correct=0
    total=0
    for nlq,gtq in zip(df['question_refine'],df['sql']):
        natural_language_query = nlq
        ground_truth_query = gtq

        # Get the predicted SparQL query
        structured = get_structured_input(nlq)
        predicted_query = chat_with_mimic4(structured)

        # Compare the predicted and ground truth queries
        print(f"Natural Language Query: {natural_language_query}")
        print(f"Predicted SparQL Query: {predicted_query}")
        print(f"Ground Truth SQL Query: {ground_truth_query}")

        correct+=compare_queries(predicted_query, ground_truth_query,thres)
        total+=1
        print("-" * 50)
        print(f"Accuracy: {correct/total}")
    return correct/total

# Run the evaluation
evaluate_on_test_data()

Streaming output truncated to the last 5000 lines.
FROM patients
JOIN diagnoses_icd ON patients.subject_id = diagnoses_icd.subject_id
WHERE patients.birth_date > DATE_SUB(CURDATE(), INTERVAL 31 YEAR)
AND diagnoses_icd.icd9_code = '70721';
Ground Truth SQL Query: SELECT COUNT ( DISTINCT DEMOGRAPHIC."SUBJECT_ID" ) FROM DEMOGRAPHIC INNER JOIN DIAGNOSES on DEMOGRAPHIC.HADM_ID = DIAGNOSES.HADM_ID WHERE DEMOGRAPHIC."AGE" < "31" AND DIAGNOSES."ICD9_CODE" = "70721"
Ground truth keywords: ['select', 'count', 'distinct', 'demographic', 'demographic', 'inner', 'join', 'diagnoses', 'demographic', 'age', '31', 'diagnoses', '70721']
Predicted keywords: ['select', 'count', 'patients', 'join', 'curdate', 'interval', '31', 'year']
Common keywords: {'select', '31', 'count', 'join'}
--------------------------------------------------
Accuracy: 0.4483695652173913
```sql
SELECT COUNT(DISTINCT subject_id) AS patient_count
FROM mimiciv.patients p
JOIN mimiciv.diagnoses d ON p.subject_id = d.subject_id
WHERE p

0.474

In [ ]:
#0.3
def compare_queries(predicted_query, ground_truth_query,thres=0.3):
    """Compares the predicted SparQL query with the ground truth SQL query.

    For simplicity, this function only checks if the predicted query contains
    some keywords from the ground truth query. A more robust comparison
    could be implemented using a dedicated query comparison library.
    """

    ground_truth_keywords = extract_keywords_nltk(ground_truth_query)
    predicted_keywords = extract_keywords_nltk(predicted_query)

    common_keywords = set(ground_truth_keywords) & set(predicted_keywords)

    # Print for analysis
    print("Ground truth keywords:", ground_truth_keywords)
    print("Predicted keywords:", predicted_keywords)
    print("Common keywords:", common_keywords)


    # Simple keyword-based comparison for demonstration
    if len(common_keywords) > thres*len(set(common_keywords).union(set(ground_truth_keywords)) ):
        return 1
    else:
        return 0

def evaluate_on_test_data(file_path="/content/mimic-sparql/dataset/mimicsql/mimicsql_natural/test.json",thres=0.3):
    """Evaluates `chat_with_mimic4` on the test data in `test.json`."""
    data = []
    with open("/content/mimic-sparql/dataset/mimicsql/mimicsql_natural/test.json", "r") as f:
      for line in f:
        data.append(json.loads(line))
    df = pd.DataFrame(data)
    correct=0
    total=0
    for nlq,gtq in zip(df['question_refine'],df['sql']):
        natural_language_query = nlq
        ground_truth_query = gtq

        # Get the predicted SparQL query
        structured = get_structured_input(nlq)
        predicted_query = chat_with_mimic4(structured)

        # Compare the predicted and ground truth queries
        print(f"Natural Language Query: {natural_language_query}")
        print(f"Predicted SparQL Query: {predicted_query}")
        print(f"Ground Truth SQL Query: {ground_truth_query}")

        correct+=compare_queries(predicted_query, ground_truth_query,thres)
        total+=1
        print("-" * 50)
        print(f"Accuracy: {correct/total}")
    return correct/total

In [ ]:
evaluate_on_test_data(thres=0.3)

Natural Language Query: how many patients were born before the year 2060?
Predicted SparQL Query: SELECT COUNT(*) 
FROM patients 
WHERE dob < '2060-01-01';
Ground Truth SQL Query: SELECT COUNT ( DISTINCT DEMOGRAPHIC."SUBJECT_ID" ) FROM DEMOGRAPHIC WHERE DEMOGRAPHIC."DOB_YEAR" < "2060"
Ground truth keywords: ['select', 'count', 'distinct', 'demographic', 'demographic', 'demographic', '2060']
Predicted keywords: ['select', 'count', 'patients', 'dob']
Common keywords: {'select', 'count'}
--------------------------------------------------
Accuracy: 1.0
Natural Language Query: how many patients had the diagnosis icd9 code 53190?
Predicted SparQL Query: SELECT COUNT(DISTINCT subject_id) AS patient_count
FROM diagnoses_icd
WHERE icd9_code = '531.90';
Ground Truth SQL Query: SELECT COUNT ( DISTINCT DEMOGRAPHIC."SUBJECT_ID" ) FROM DEMOGRAPHIC INNER JOIN DIAGNOSES on DEMOGRAPHIC.HADM_ID = DIAGNOSES.HADM_ID WHERE DIAGNOSES."ICD9_CODE" = "53190"
Ground truth keywords: ['select', 'count', 'distinct

KeyboardInterrupt: 

In [ ]:
#4o 0.5
evaluate_on_test_data(thres=0.5)

Streaming output truncated to the last 5000 lines.
WHERE (p.ethnicity LIKE '%Black%' OR p.ethnicity LIKE '%Haitian%')
AND a.admittime < '2124-01-01';
Ground Truth SQL Query: SELECT COUNT ( DISTINCT DEMOGRAPHIC."SUBJECT_ID" ) FROM DEMOGRAPHIC WHERE DEMOGRAPHIC."ETHNICITY" = "BLACK/HAITIAN" AND DEMOGRAPHIC."ADMITYEAR" < "2124"
Ground truth keywords: ['select', 'count', 'distinct', 'demographic', 'demographic', 'demographic', 'ethnicity', 'demographic', 'admityear', '2124']
Predicted keywords: ['select', 'count', 'admissions', 'join', 'patients', 'p', 'like', 'black', 'like', 'haitian']
Common keywords: {'select', 'count'}
--------------------------------------------------
Accuracy: 0.4243369734789392
Natural Language Query: how many patients are with asian ethnicity and admitted before the year 2110?
Predicted SparQL Query: SELECT COUNT(DISTINCT patients.subject_id) AS asian_patients_before_2110
FROM patients
JOIN admissions ON patients.subject_id = admissions.subject_id
WHERE patients.e

0.457

In [ ]:
#4o 0.6
evaluate_on_test_data(thres=0.6)

Streaming output truncated to the last 5000 lines.
FROM patients AS p
JOIN admissions AS a ON p.subject_id = a.subject_id
JOIN diagnoses_icd AS d ON a.hadm_id = d.hadm_id
WHERE p.ethnicity IN ('BLACK/HAITIAN')
AND d.icd9_code = '2948';
Ground Truth SQL Query: SELECT COUNT ( DISTINCT DEMOGRAPHIC."SUBJECT_ID" ) FROM DEMOGRAPHIC INNER JOIN DIAGNOSES on DEMOGRAPHIC.HADM_ID = DIAGNOSES.HADM_ID WHERE DEMOGRAPHIC."ETHNICITY" = "BLACK/HAITIAN" AND DIAGNOSES."ICD9_CODE" = "2948"
Ground truth keywords: ['select', 'count', 'distinct', 'demographic', 'demographic', 'inner', 'join', 'diagnoses', 'demographic', 'ethnicity', 'diagnoses', '2948']
Predicted keywords: ['select', 'count', 'distinct', 'patients', 'p', 'join', 'admissions', 'join']
Common keywords: {'select', 'count', 'join', 'distinct'}
--------------------------------------------------
Accuracy: 0.27527216174183516
Natural Language Query: how many of the asian patients had unspecified neutropenia?
Predicted SparQL Query: SELECT COUNT(DIS

0.284

In [ ]:
#4o 0.4
evaluate_on_test_data(thres=0.4)

Streaming output truncated to the last 5000 lines.
Predicted keywords: ['select', 'count', 'admissions', 'join', 'patients', 'like']
Common keywords: {'select', 'count'}
--------------------------------------------------
Accuracy: 0.7254290171606864
Natural Language Query: how many patients are with asian ethnicity and admitted before the year 2110?
Predicted SparQL Query: SELECT COUNT(DISTINCT subject_id) AS num_patients
FROM patients
WHERE ethnicity LIKE '%Asian%'
AND EXTRACT(YEAR FROM admittime) < 2110;
```
Ground Truth SQL Query: SELECT COUNT ( DISTINCT DEMOGRAPHIC."SUBJECT_ID" ) FROM DEMOGRAPHIC WHERE DEMOGRAPHIC."ETHNICITY" = "ASIAN" AND DEMOGRAPHIC."ADMITYEAR" < "2110"
Ground truth keywords: ['select', 'count', 'distinct', 'demographic', 'demographic', 'demographic', 'ethnicity', 'asian', 'demographic', 'admityear', '2110']
Predicted keywords: ['select', 'count', 'distinct', 'patients', 'ethnicity', 'like', 'asian', 'extract', 'year', 'admittime', '2110']
Common keywords: {'ethn

0.744